In [1]:
# imports 
import pandas as pd # to read the csv files 
import os.path
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# need to allow for the memory limit to be able to grow (?) https://www.tensorflow.org/guide/gpu
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
# maybe we should use IMG instead...
# thanks so much to Alex Kyllo for pointing this out to us!
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
rescale_datagen = ImageDataGenerator(rescale=1./255)
train_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/train',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)
validation_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/valid',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)

Found 99999 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [4]:
# this time around, we'll play around with the BN layer order, as well as using 'selu' as our activation function, which requires 
# LeCun initialization 
vgg_v2 = models.Sequential()
vgg_v2.add(layers.InputLayer(input_shape=(256, 256, 3)))
for i in range(2):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(64, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
for i in range(2):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(128, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
for i in range(3):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(256, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
for i in range(3):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(512, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
vgg_v2.add(layers.Flatten())
for i in range(2):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Dense(4086, kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.BatchNormalization())
vgg_v2.add(layers.Dense(2, activation='softmax'))

In [5]:
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [6]:
vgg_v2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])
vgg_v2_history = vgg_v2.fit(train_generator, 
                                  epochs=50, 
                                  validation_data=validation_generator, 
                                  callbacks=[
                                      tf.keras.callbacks.ModelCheckpoint('models/vgg_v2', save_best_only=True),
                                      tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True),
                                      tf.keras.callbacks.TensorBoard(get_run_logdir())
                                  ])

Epoch 1/50
3125/3125 [==============================] - ETA: 0s - loss: 0.3024 - accuracy: 0.8777WARNING:tensorflow:From C:\Users\valen\anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/vgg_v2\assets
3125/3125 [==============================] - 971s 311ms/step - loss: 0.3024 - accuracy: 0.8777 - val_loss: 0.1782 - val_accuracy: 0.9300
Epoch 2/50
3125/3125 [==============================] - 965s 309ms/step - loss: 0.0872 - accuracy: 0.9685 - val_loss: 0.0642 - val_accuracy: 0.9769
Epoch 3/50
3125/3125 [==============================] - 956s 306ms/step - loss: 0.0585 - accuracy: 0.9792 - val_loss: 0.0652 - val_accuracy: 0.9786
Epoch 4/50
3125/3125 [==============================]

3125/3125 [==============================] - 1377s 441ms/step - loss: 0.0035 - accuracy: 0.9988 - val_loss: 0.0087 - val_accuracy: 0.9970
Epoch 43/50
3125/3125 [==============================] - 1168s 374ms/step - loss: 0.0033 - accuracy: 0.9989 - val_loss: 0.0110 - val_accuracy: 0.9966
Epoch 44/50
3125/3125 [==============================] - 937s 300ms/step - loss: 0.0034 - accuracy: 0.9988 - val_loss: 0.0107 - val_accuracy: 0.9972
Epoch 45/50
3125/3125 [==============================] - 937s 300ms/step - loss: 0.0037 - accuracy: 0.9987 - val_loss: 0.0101 - val_accuracy: 0.9965
Epoch 46/50
3125/3125 [==============================] - 937s 300ms/step - loss: 0.0026 - accuracy: 0.9990 - val_loss: 0.0127 - val_accuracy: 0.9966
Epoch 47/50
3125/3125 [==============================] - 936s 300ms/step - loss: 0.0034 - accuracy: 0.9988 - val_loss: 0.0112 - val_accuracy: 0.9965
Epoch 48/50
3125/3125 [==============================] - 945s 302ms/step - loss: 0.0031 - accuracy: 0.9990 - val_los

In [7]:
test_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/test',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)

Found 20000 images belonging to 2 classes.


In [8]:
vgg_v2.evaluate(test_generator, batch_size=32)

625/625 [==============================] - 114s 183ms/step - loss: 0.0097 - accuracy: 0.9977


[0.009659171104431152, 0.9976500272750854]